In [ ]:
!pip freeze > requeriments.txt

In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [ ]:
import h5py
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
#import tensorflow as tf
#import sklearn.metrics
import pickle

from sklearn.preprocessing import OneHotEncoder

from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers, Model, optimizers


# Collecting Data

In [ ]:
filename = "/content/drive/MyDrive/Colab Notebooks/UTEC_Sec_Project/data/VDISC_train.hdf5"
data_train = {}

with h5py.File(filename, "r") as f:
    column_names = list(f.keys())
    for column in column_names:
        data_train[column] = list(f[column])

In [ ]:
df_train = pd.DataFrame(data_train)
#df_train = df_train.head(100000) #optional, comment in case you want the whole data
print(df_train.shape)
df_train.head()

(1019471, 6)


,CWE-119,CWE-120,CWE-469,CWE-476,CWE-other,functionSource
0,False,False,False,False,False,"b'clear_area(int startx, int starty, int xsize..."
1,False,False,False,False,False,b'ReconstructDuList(Statement* head)\n{\n S...
2,False,False,False,False,False,b'free_speaker(void)\n{\n if(Lengths)\n ...
3,False,False,False,False,False,b'mlx4_register_device(struct mlx4_dev *dev)\n...
4,True,True,False,False,True,"b'Parse_Env_Var(void)\n{\n char *p = getenv(""..."


In [ ]:
df_train_cwe = df_train[df_train["CWE-119"] | df_train["CWE-120"] | df_train["CWE-469"] | df_train["CWE-476"] | df_train["CWE-other"]]

In [ ]:
df_train_clean = df_train.drop(df_train_cwe.index)

In [ ]:
df_train_cwe.shape[0] / df_train_clean.shape[0] *100

6.9113129963599835

In [ ]:
df_train_clean_sample = df_train_clean.sample(df_train_cwe.shape[0], random_state=43) #get clean data, code withou vulnerabilities
df_train = pd.concat([df_train_cwe, df_train_clean_sample]) # combine 
df_train = df_train.sample(frac=1).reset_index(drop=True) #shufle data
df_train

,CWE-119,CWE-120,CWE-469,CWE-476,CWE-other,functionSource
0,False,False,False,False,False,b'isSocksProxy()\n{\n return (QNetworkProxy...
1,False,False,False,False,False,"b'command_invoke(struct command_base* cbase, s..."
2,False,False,False,False,False,"b'PyModule_AddObject(PyObject *m, const char *..."
3,False,False,False,False,True,"b'genbank_to_alma(inf, outf)\n char *inf..."
4,True,True,False,False,False,b'mode_select_prepick (struct fujitsu *s)\n{\n...
...,...,...,...,...,...,...
131803,False,False,False,False,True,"b'sqliteOsOpenExclusive(const char *zFilename,..."
131804,False,False,False,False,False,b'qColor() const\n{\n // start with sane co...
131805,False,True,False,False,False,"b'handle_topology_map(struct fw_card *card, st..."
131806,False,False,False,False,False,b'archive_write_set_format_by_name(struct arch...


In [ ]:
del(data_train) # free up space 

In [ ]:
filename_val = "/content/drive/MyDrive/Colab Notebooks/UTEC_Sec_Project/data/VDISC_validate.hdf5"
data_val = {}

with h5py.File(filename_val, "r") as f:
    column_names = list(f.keys())
    for column in column_names:
        data_val[column] = list(f[column])

In [ ]:
df_val = pd.DataFrame(data_val)
#df_val = df_val.head(10000) #optional, comment in case you want the whole data
print(df_val.shape)
df_val.head()

(127476, 6)


,CWE-119,CWE-120,CWE-469,CWE-476,CWE-other,functionSource
0,False,False,False,False,False,b'gwy_resource_class_mkdir(GwyResourceClass *k...
1,False,False,False,False,False,b'SetIgnoredFields( const char **papszFields )...
2,False,False,False,False,False,b'timeoutProtoDisplays(void)\n{\n struct pr...
3,False,False,False,False,False,"b'SelectViewsOf (GraphicComp* comp, Editor* ed..."
4,False,False,False,False,False,b'multi_get_bool(bool &value)\n{\n\tif (!Multi...


In [ ]:
df_val_cwe = df_val[df_val["CWE-119"] | df_val["CWE-120"] | df_val["CWE-469"] | df_val["CWE-476"] | df_val["CWE-other"]]

In [ ]:
df_val_clean = df_val.drop(df_val_cwe.index)

In [ ]:
df_val_clean_sample = df_val_clean.sample(df_val_cwe.shape[0], random_state=43) #get clean data, code withou vulnerabilities
df_val = pd.concat([df_val_cwe, df_val_clean_sample]) # combine 
df_val = df_val.sample(frac=1).reset_index(drop=True) #shufle data
df_val

,CWE-119,CWE-120,CWE-469,CWE-476,CWE-other,functionSource
0,True,True,False,False,True,"b'uncompress_to_file(char *f_src, char *f_targ..."
1,True,True,False,False,False,b'ipmi_mc_shutoff_watchdog(struct ipmi_intf * ...
2,False,True,False,False,False,"b'svdNewDMat(int rows, int cols) {\n int i;\n..."
3,False,False,False,False,False,b'store_sampling_down_factor(struct dbs_data *...
4,False,True,False,False,False,"b""esl_sq_SetDesc(ESL_SQ *sq, const char *desc)..."
...,...,...,...,...,...,...
16503,True,True,False,False,False,"b'plugin_emit_keypress (session *sess, unsigne..."
16504,False,False,False,False,False,b'batrider_create_tilemaps_0(void)\r\n{\r\n\tt...
16505,False,False,False,True,False,b'list_all_channels(struct Client *source_p)\n...
16506,True,True,False,False,False,"b""adc_msg_get_argument_index(struct adc_messag..."


In [ ]:
del(data_val) # free up space 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# EDA 

In [ ]:
df_train.isnull().sum()

CWE-119           0
CWE-120           0
CWE-469           0
CWE-476           0
CWE-other         0
functionSource    0
dtype: int64

In [ ]:
X_train = df_train["functionSource"]
y_train = df_train.drop(columns="functionSource")
X_train.shape, y_train.shape

((131808,), (131808, 5))

In [ ]:
X_val = df_val["functionSource"]
y_val = df_val.drop(columns="functionSource")
X_val.shape, y_val.shape

((16508,), (16508, 5))

# Preprocessing

In [ ]:
# Set the global value
WORDS_SIZE=10000
INPUT_SIZE=500
NUM_CLASSES=2
MODEL_NUM=0
EPOCHS=10

## Scaling Output (CWE)

In [ ]:
ohe = OneHotEncoder(drop="if_binary", sparse=False)
y_train_preprocessed = ohe.fit_transform(y_train)
y_train_preprocessed

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       ...,
       [0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])

In [ ]:
y_val_preprocessed = ohe.transform(y_val)
y_val_preprocessed

array([[1., 1., 0., 0., 1.],
       [1., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       ...,
       [0., 0., 0., 1., 0.],
       [1., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.]])

In [ ]:
X_train = X_train.apply(lambda x : x.decode("utf-8"))
X_val = X_val.apply(lambda x : x.decode("utf-8"))

## Tokenization

## For Training Dataset

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(list(X_train))
len(tokenizer.word_counts)

332307

In [ ]:
tokenizer.num_words = WORDS_SIZE
tokenizer.num_words

10000

In [ ]:
list_tokenized_train = tokenizer.texts_to_sequences(X_train)

In [ ]:
X_train_pad = pad_sequences(list_tokenized_train,  maxlen=INPUT_SIZE, padding='post', dtype = float)

In [ ]:
y_train_preprocessed.shape

(131808, 5)

In [ ]:
y_train=[]
for col in range(5):
    y_train.append(to_categorical(y_train_preprocessed[:,col], num_classes=NUM_CLASSES).astype(np.int64))
len(y_train)

5

## For Validation Dataset

In [ ]:
list_tokenized_val = tokenizer.texts_to_sequences(X_val)
X_val_pad = pad_sequences(list_tokenized_val,  maxlen=INPUT_SIZE, padding='post', dtype = float)

In [ ]:
y_val=[]
for col in range(5):
    y_val.append(to_categorical(y_val_preprocessed[:,col], num_classes=NUM_CLASSES).astype(np.int64))
len(y_val)

5

# Modeling

In [ ]:
random_weights = np.random.normal(size=(WORDS_SIZE, 13),scale=0.01)

In [ ]:
# Must use non-sequential model building to create branches in the output layer
inp_layer = layers.Input(shape=(INPUT_SIZE,))
mid_layers = layers.Embedding(input_dim = WORDS_SIZE,
                                    output_dim = 13,
                                    weights=[random_weights],
                                    input_length = INPUT_SIZE)(inp_layer)
mid_layers = layers.Convolution1D(filters=512, kernel_size=(9), padding='same', activation='relu')(mid_layers)
mid_layers = layers.MaxPool1D(pool_size=5)(mid_layers)
mid_layers = layers.Dropout(0.5)(mid_layers)
mid_layers = layers.Flatten()(mid_layers)
mid_layers = layers.Dense(64, activation='relu')(mid_layers)
mid_layers = layers.Dense(16, activation='relu')(mid_layers)
output1 = layers.Dense(2, activation='softmax')(mid_layers)
output2 = layers.Dense(2, activation='softmax')(mid_layers)
output3 = layers.Dense(2, activation='softmax')(mid_layers)
output4 = layers.Dense(2, activation='softmax')(mid_layers)
output5 = layers.Dense(2, activation='softmax')(mid_layers)
model = Model(inp_layer,[output1,output2,output3,output4,output5])

# Define custom optimizers
adam = optimizers.Adam(learning_rate=0.005, beta_1=0.9, beta_2=0.999, epsilon=1, decay=0.0, amsgrad=False)

## Compile model with metrics
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=[tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
print("CNN model built: ")
model.summary()

CNN model built: 
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 500)]        0           []                               
                                                                                                  
 embedding (Embedding)          (None, 500, 13)      130000      ['input_1[0][0]']                
                                                                                                  
 conv1d (Conv1D)                (None, 500, 512)     60416       ['embedding[0][0]']              
                                                                                                  
 max_pooling1d (MaxPooling1D)   (None, 100, 512)     0           ['conv1d[0][0]']                 
                                                                            

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

In [ ]:
class_weights = [{0: 1., 1: 5.},{0: 1., 1: 5.},{0: 1., 1: 5.},{0: 1., 1: 5.},{0: 1., 1: 5.}]

history = model.fit(x = X_train_pad,
          y = [y_train[0], y_train[1], y_train[2], y_train[3], y_train[4]],
          validation_data = (X_val_pad, [y_val[0], y_val[1], y_val[2], y_val[3], y_val[4]]),
          epochs = 100,
          batch_size = 128,
          verbose =2,
          callbacks=[callback]
          #class_weight= dict(enumerate(class_weights))
          )

Epoch 1/100
1030/1030 - 44s - loss: 2.1218 - dense_2_loss: 0.4555 - dense_3_loss: 0.6055 - dense_4_loss: 0.1802 - dense_5_loss: 0.3237 - dense_6_loss: 0.5569 - dense_2_precision: 0.8533 - dense_2_recall: 0.8533 - dense_3_precision: 0.7114 - dense_3_recall: 0.7114 - dense_4_precision: 0.9772 - dense_4_recall: 0.9772 - dense_5_precision: 0.9252 - dense_5_recall: 0.9252 - dense_6_precision: 0.7498 - dense_6_recall: 0.7498 - val_loss: 1.8488 - val_dense_2_loss: 0.4070 - val_dense_3_loss: 0.5830 - val_dense_4_loss: 0.0781 - val_dense_5_loss: 0.2601 - val_dense_6_loss: 0.5206 - val_dense_2_precision: 0.8535 - val_dense_2_recall: 0.8535 - val_dense_3_precision: 0.7123 - val_dense_3_recall: 0.7123 - val_dense_4_precision: 0.9847 - val_dense_4_recall: 0.9847 - val_dense_5_precision: 0.9268 - val_dense_5_recall: 0.9268 - val_dense_6_precision: 0.7832 - val_dense_6_recall: 0.7832 - 44s/epoch - 43ms/step
Epoch 2/100
1030/1030 - 29s - loss: 1.8341 - dense_2_loss: 0.4038 - dense_3_loss: 0.5784 - den

In [ ]:
model.save("/content/drive/MyDrive/Colab Notebooks/UTEC_Sec_Project/data/model_clean_v1.hdf5")

In [ ]:
new_model = tf.keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/UTEC_Sec_Project/data/model_clean_v1.hdf5')
new_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 500)]        0           []                               
                                                                                                  
 embedding (Embedding)          (None, 500, 13)      130000      ['input_1[0][0]']                
                                                                                                  
 conv1d (Conv1D)                (None, 500, 512)     60416       ['embedding[0][0]']              
                                                                                                  
 max_pooling1d (MaxPooling1D)   (None, 100, 512)     0           ['conv1d[0][0]']                 
                                                                                              

# Testing

In [ ]:
filename_test = "/content/drive/MyDrive/Colab Notebooks/UTEC_Sec_Project/data/VDISC_test.hdf5"
data_test = {}

with h5py.File(filename_test, "r") as f:
    column_names = list(f.keys())
    for column in column_names:
        data_test[column] = list(f[column])

In [ ]:
df_test = pd.DataFrame(data_test)
print(df_test.shape)
df_test.head()

(127419, 6)


,CWE-119,CWE-120,CWE-469,CWE-476,CWE-other,functionSource
0,False,False,False,False,False,b'default_event_handler( \n GuiWidget *widg...
1,False,False,False,False,False,"b'krb5_krbhst_init_flags(krb5_context context,..."
2,False,False,False,False,False,b'swap_info_get(swp_entry_t entry)\n{\n\tstruc...
3,False,False,False,False,False,"b'parseattribs4(char *&c, const Vec4 &ival = V..."
4,False,False,False,False,False,b'generateExecCode(CompileState* comp)\n{\n ...
